In [1]:
import pandas as pd
data_train = pd.read_csv('sarcasm_tam_train.csv')
data_val = pd.read_csv('sarcasm_tam_dev.csv')
data_test = pd.read_csv('sarcasm_tam_test_without_labels.csv')

In [27]:
text_train = data_train['Text']
text_val = data_val['Text']

class_label_train = data_train['labels']
class_label_val = data_val['labels']

text_test = data_test['Text']

In [28]:
class_label_train.value_counts(), class_label_val.value_counts()

(labels
 Non-sarcastic    21740
 Sarcastic         7830
 Name: count, dtype: int64,
 labels
 Non-sarcastic    4630
 Sarcastic        1706
 Name: count, dtype: int64)

In [29]:
import numpy as np
classes_list = ["Non-sarcastic", "Sarcastic"]
label_index_train = class_label_train.apply(classes_list.index)
label_index_val = class_label_val.apply(classes_list.index)

In [30]:
import numpy as np
import ktrain
from ktrain import text

In [7]:
# MODEL_NAME = 'distilbert-base-multilingual-cased'
MODEL_NAME = 'google-bert/bert-base-multilingual-cased'
t = text.Transformer(MODEL_NAME, maxlen=30, classes=classes_list)
trn = t.preprocess_train(np.array(text_train), np.array(class_label_train))
val = t.preprocess_test(np.array(text_val), np.array(class_label_val))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

model.safetensors:  80%|#######9  | 571M/714M [00:00<?, ?B/s]

preprocessing train...
language: ta
train sequence lengths:
	mean : 10
	95percentile : 23
	99percentile : 43


c:\Users\nimmi\anaconda3\Lib\site-packages\ktrain\utils.py:744: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: ta
test sequence lengths:
	mean : 10
	95percentile : 21
	99percentile : 41


In [8]:
learner.fit_onecycle(5e-5, 10) 



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/10
925/925 [==============================] - 18014s 19s/step - loss: 0.5059 - accuracy: 0.7464 - val_loss: 0.4556 - val_accuracy: 0.7737
Epoch 2/10
925/925 [==============================] - 9043s 10s/step - loss: 0.4333 - accuracy: 0.7896 - val_loss: 0.4448 - val_accuracy: 0.7693
Epoch 3/10
925/925 [==============================] - 5201s 6s/step - loss: 0.3748 - accuracy: 0.8294 - val_loss: 0.4439 - val_accuracy: 0.7921
Epoch 4/10
925/925 [==============================] - 4280s 5s/step - loss: 0.3201 - accuracy: 0.8614 - val_loss: 0.4710 - val_accuracy: 0.7907
Epoch 5/10
925/925 [==============================] - 4577s 5s/step - loss: 0.2560 - accuracy: 0.8971 - val_loss: 0.5021 - val_accuracy: 0.7850
Epoch 6/10
925/925 [==============================] - 4136s 4s/step - loss: 0.1980 - accuracy: 0.9241 - val_loss: 0.5388 - val_accuracy: 0.7943
Epoch 7/10
925/925 [==============================] - 4261s 5s/step - 

In [9]:
learner.validate(class_names=t.get_classes()) 

198/198 [==============================] - 219s 1s/step
               precision    recall  f1-score   support

Non-sarcastic       0.84      0.90      0.87      4630
    Sarcastic       0.65      0.52      0.58      1706

     accuracy                           0.80      6336
    macro avg       0.74      0.71      0.72      6336
 weighted avg       0.79      0.80      0.79      6336



array([[4153,  477],
       [ 817,  889]], dtype=int64)

In [10]:
learner.view_top_losses(n=1, preproc=t)

198/198 [==============================] - 206s 1s/step
----------
id:438 | loss:6.49 | true:Sarcastic | pred:Non-sarcastic)



In [11]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [12]:
predictor.predict(text_test[0])

'Non-sarcastic'

In [13]:
predictor.predict_proba(text_test[0])

array([0.99848384, 0.00151622], dtype=float32)

In [14]:
predictor.get_classes()

['Non-sarcastic', 'Sarcastic']

In [15]:
predictor.explain(text_test[0])

In [16]:
text_test[1]

'விழுப்புரம்  வன்னிய கவுண்டர் சார்பாக வாழ்த்துக்கள்'

In [17]:
predictor.save('Google_Bert_TAMIL_SARCASM_DETECTION')

In [18]:
reloaded_predictor = ktrain.load_predictor('Google_Bert_TAMIL_SARCASM_DETECTION')

In [19]:
predict = reloaded_predictor.predict(text_test.values)

In [20]:
data_test['label'] = predict

In [21]:
new_data_test = data_test[['ID','label']]

In [22]:
import csv
new_data_test.to_csv("Tamil_google_bert_sarcasm_detection.tsv", sep="\t", quoting=csv.QUOTE_NONE)

In [23]:
check_submission = pd.read_csv("Tamil_google_bert_sarcasm_detection.tsv", sep="\t")

In [24]:
check_submission

,Unnamed: 0,ID,label
0,0,Id_01,Non-sarcastic
1,1,Id_02,Non-sarcastic
2,2,Id_03,Non-sarcastic
3,3,Id_04,Non-sarcastic
4,4,Id_05,Sarcastic
...,...,...,...
6333,6333,Id_6334,Non-sarcastic
6334,6334,Id_6335,Sarcastic
6335,6335,Id_6336,Non-sarcastic
6336,6336,Id_6337,Non-sarcastic


In [25]:
check_submission['label'].value_counts()

label
Non-sarcastic    4913
Sarcastic        1425
Name: count, dtype: int64

In [26]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_75 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 177854978 (678.46 MB)
Trainable params: 177854978 (678.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
